## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-10-20-51-00 +0000,wsj,Opinion | Trump Plans to Give Your 401(k) a Boost,https://www.wsj.com/opinion/trump-plans-to-giv...
1,2025-08-10-20-45-21 +0000,nypost,MTA touts subway system safety — just hours af...,https://nypost.com/2025/08/10/us-news/mta-tout...
2,2025-08-10-20-42-37 +0000,nypost,US tourist shot dead during Puerto Rico trip t...,https://nypost.com/2025/08/10/us-news/us-touri...
3,2025-08-10-20-39-56 +0000,startribune,Luke Keaschall’s walk-off homer give Twins 5-3...,https://www.startribune.com/minnesota-twins-lu...
4,2025-08-10-20-39-00 +0000,wsj,Opinion | A Supreme Court Coup d’Etat in Brazil,https://www.wsj.com/opinion/a-supreme-court-co...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,25
41,putin,14
170,ukraine,12
119,gaza,9
114,new,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
76,2025-08-10-14-15-48 +0000,nyt,"In a Trump-Putin Summit, Ukraine Fears Losing ...",https://www.nytimes.com/2025/08/10/us/politics...,67
60,2025-08-10-16-41-53 +0000,nyt,Dmitri Kozak Was a Key Putin Aide. He Lost Pow...,https://www.nytimes.com/2025/08/10/world/europ...,66
155,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...,64
19,2025-08-10-19-52-29 +0000,nypost,Vance says only Trump can decide when to bring...,https://nypost.com/2025/08/10/us-news/vance-sa...,64
59,2025-08-10-16-46-36 +0000,nyt,Ukraine and Europe Project United Front Ahead ...,https://www.nytimes.com/2025/08/10/world/europ...,63


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
76,67,2025-08-10-14-15-48 +0000,nyt,"In a Trump-Putin Summit, Ukraine Fears Losing ...",https://www.nytimes.com/2025/08/10/us/politics...
14,41,2025-08-10-20-08-00 +0000,wsj,Israel’s New War Plan Includes Fighting in Gaz...,https://www.wsj.com/world/middle-east/israels-...
155,39,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...
101,30,2025-08-10-10-00-00 +0000,latimes,"Arellano: When the going gets tough for L.A., ...",https://www.latimes.com/california/story/2025-...
170,26,2025-08-09-22-47-00 +0000,wsj,Adriana Kugler hasn’t said why she is leaving ...,https://www.wsj.com/economy/central-banking/su...
31,25,2025-08-10-19-05-13 +0000,nypost,Latest NYC subway shove victim is ER doc who w...,https://nypost.com/2025/08/10/us-news/nyc-subw...
160,24,2025-08-09-23-34-20 +0000,nypost,White House weighs inviting Zelensky to Trump’...,https://nypost.com/2025/08/09/us-news/white-ho...
173,22,2025-08-09-21-47-06 +0000,nypost,"US, Qatari mediators try to shape last-minute ...",https://nypost.com/2025/08/09/world-news/us-qa...
93,21,2025-08-10-10-00-00 +0000,latimes,"California took center stage in ICE raids, but...",https://www.latimes.com/california/story/2025-...
172,19,2025-08-09-22-09-21 +0000,nypost,Russian drone strikes kill at least 8 civilian...,https://nypost.com/2025/08/09/world-news/mosco...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
